# Motivation

For my next piece of work, I need to determine the distances between multivariate Gaussian distributions to obtain a measure of the separation of the distributions, i.e. the average distance between a set of Gaussians. Although used often, the Kullback-Leibler (or KL) divergence is not a true metric and so is not quite right for this task. Fortunately, the (square root) of the Jensen-Shannon (or JS) divergence, based on KL, is a metric.

I wish to keep the distributions defined puely in terms of their mean and covariance, and avoid sampling and making any comparisons between data points.

**Note**: A pdf of the maths part of this notebook can be found in the parent folder, and is typeset a little nicer than is possible here due to my desire to put comments between each line of maths, something that MathJax is not designed for.

# Definitions
## Multivariate Gaussian

The density function of a multivariate Gaussian can be defined as follows

\begin{equation}
	p(x) = \frac{1}{(2\pi)^{n/2} \left | \Sigma \right |^{1/2}} \exp \left ( -\frac{1}{2} (x - \mu)^{T} \Sigma^{-1} (x - \mu) \right )
\end{equation}

where $\mu$ is the mean vector of dimension $d$ and $\Sigma$ is the covariance matrix.

Taking the log of this is as follows

\begin{equation}
	\log p(x) = -\frac{n}{2} \log 2\pi - \frac{1}{2} \log | \Sigma | - \frac{1}{2}(x-\mu)^{T}\Sigma^{-1} (x - \mu)
\end{equation}

This form will be useful later...

## Kullback-Leibler (KL) Divergence

The KL divergence between two distributions P and Q is defined as

\begin{equation}
	D_{KL}(P~||~Q) = \mathbb{E}_{P} \left [ \log \frac{P}{Q} \right ]
\end{equation}

where $\mathbb{E}_{P}$ is the expectation for the distribution P.

## Jensen-Shannon (JS) Divergence

The Jensen-Shannon (JS) divergence is symmetric version of the KL Divergence, defined as follows

\begin{equation} 
	D_{JS}(P~||~Q) = \frac{1}{2} D_{KL}(P~||~M) + \frac{1}{2} D_{KL}(Q~||~M)
\end{equation}

where $M=\frac{1}{2}(P+Q)$. Given two multivariate Gaussians $P$ and $Q$, $M\sim \mathcal{N}(\mu_{M}, \Sigma_{M})$, where $\mu_{M} = \frac{1}{2}(\mu_{P} + \mu_{Q})$ and $\Sigma_{M} = \frac{1}{2}(\Sigma_{P} + \Sigma_{Q})$.

## JS Distance

To obtain a metric, we simply use $\sqrt{D_{JS}(P~||~Q)}$.

# Formulations
## The KL Divergence for a Multivariate Gaussian

$$
\newcommand\numberthis{\addtocounter{equation}{1}\tag{\theequation}}
\newcommand{\mathcomment}[1]{\shortintertext{\textit{#1:}}}
$$

So for two multivariate Gaussians, $P_1$ and $P_2$, in $\mathbb{R}^{d}$, we can reformulate KL as

\begin{align*}
	D_{KL}(P_{1}~||~P_{2}) ={}& \mathbb{E}_{P_{1}} \left [ \log P_1 - \log P_2 \right ]\\[0.6cm]
\end{align*}

If we substitute the log form of the definition of the multivariate Gaussian for $P_1$ and $P_2$ then we get:

\begin{align*}
	={}& \mathbb{E}_{P_{1}} \bigg[ \left ( -\frac{n}{2} \log 2\pi - \frac{1}{2} \log | \Sigma_1 | - \frac{1}{2}(x-\mu_1)^{T}\Sigma^{-1}_1 (x - \mu_1) \right )&\\
	{}& - \left ( -\frac{n}{2} \log 2\pi - \frac{1}{2} \log | \Sigma_2 | - \frac{1}{2}(x-\mu_2)^{T}\Sigma^{-1}_2 (x - \mu_2) \right ) \bigg]\\[0.6cm]
\end{align*}

Cancel terms

\begin{align*}
	&&&={} \mathbb{E}_{P_{1}} \left [ - \frac{1}{2} \log | \Sigma_1 | - \frac{1}{2}(x-\mu_1)^{T}\Sigma^{-1}_1 (x - \mu_1) - \frac{1}{2} \log | \Sigma_2 | - \frac{1}{2}(x-\mu_2)^{T}\Sigma^{-1}_2 (x - \mu_2) \right ]\\[0.6cm]
\end{align*}

Factor out the constant

\begin{align*}
	&={} \mathbb{E}_{P_{1}} \frac{1}{2} \left [ -\log | \Sigma_1 | - (x-\mu_1)^{T}\Sigma^{-1}_1 (x - \mu_1) - \log | \Sigma_2 | - (x-\mu_2)^{T}\Sigma^{-1}_2 (x - \mu_2) \right ]\\[0.6cm]
\end{align*}

Factor out the determinants

\begin{align*}
	={} \frac{1}{2} \log \frac{| \Sigma_2 |}{| \Sigma_1 |} + \mathbb{E}_{P_{1}} \frac{1}{2} \left [ - (x-\mu_1)^{T}\Sigma^{-1}_1 (x - \mu_1) + (x-\mu_2)^{T}\Sigma^{-1}_2 (x - \mu_2) \right ]&&\\[0.6cm]
\end{align*}

We can use the trace trick here. As the term $(x-\mu)^{T}\Sigma^{-1} (x - \mu)$ simplifies to a scalar, we can use $\mathrm{Tr}(c) = c$ combined with $\mathrm{Tr}(ABC) = \mathrm{Tr}(BCA)$ to rearrange terms

\begin{align*}
    ={}& \frac{1}{2} \log \frac{| \Sigma_2 |}{| \Sigma_1 |} + \frac{1}{2} \mathbb{E}_{P_{1}} \left [ - \mathrm{Tr}(\Sigma^{-1}_1 (x - \mu_1)(x-\mu_1)^{T}) + (x-\mu_2)^{T}\Sigma^{-1}_2 (x - \mu_2) \right ]\\[0.6cm]
\end{align*}

We can simplify $\mathbb{E}_{P_{1}} \left [ (x - \mu_1)(x-\mu_1)^{T} \right ]$ as it is just the covariance $(\Sigma_1)$

\begin{align*}
    ={}& \frac{1}{2} \log \frac{| \Sigma_2 |}{| \Sigma_1 |} - \frac{1}{2} \mathrm{Tr}(\Sigma^{-1}_1 \Sigma_1) + \frac{1}{2} \mathbb{E}_{P_{1}} \left [(x-\mu_2)^{T}\Sigma^{-1}_2 (x - \mu_2) \right ]&&&&&&\\[0.6cm]
\end{align*}

The $\mathrm{Tr}(\Sigma^{-1}_1 \Sigma_1)$ term simplifies to $\mathrm{Tr}(I_d)=d$

\begin{align*}
    ={}& \frac{1}{2} \log \frac{| \Sigma_2 |}{| \Sigma_1 |} - \frac{1}{2} d + \frac{1}{2} \mathbb{E}_{P_{1}} \left [(x-\mu_2)^{T}\Sigma^{-1}_2 (x - \mu_2) \right ]&&&&&&&&\\[0.6cm]
\end{align*}

Using equation 380 from the matrix cookbook $\Big( E\left [ (\mathbf{x}-\mathbf{m}^{\prime})^{T} \mathbf{A} (\mathbf{x}-\mathbf{m}^{\prime}) \right ] = (\mathbf{m}-\mathbf{m}^{\prime})^{T} \mathbf{A} (\mathbf{m}-\mathbf{m}^{\prime}) + \mathrm{Tr}(\mathbf{A}\mathbf{\Sigma}) \Big)$

\begin{align*}
    ={}& \frac{1}{2} \log \frac{| \Sigma_2 |}{| \Sigma_1 |} - \frac{1}{2} d + \frac{1}{2} (\mu_1 - \mu_2)^{T} \Sigma^{-1}_2 (\mu_1 - \mu_2) + \mathrm{Tr}(\Sigma^{-1}_2 \Sigma_1)&&&&&\\[0.6cm]
\end{align*}

We can swap the $\mu_2$ and $\mu_1$ terms around and factor out the $\frac{1}{2}$ to get the final result

\begin{align*}
    D_{KL}(P_{1}~||~P_{2}) ={}& \frac{1}{2} \left ( \log \frac{| \Sigma_2 |}{| \Sigma_1 |} - d + (\mu_2 - \mu_1)^{T} \Sigma^{-1}_2 (\mu_2 - \mu_1) + \mathrm{Tr}(\Sigma^{-1}_2 \Sigma_1) \right )&&&&&&&&&&&&&&&&&&
\end{align*}

## The JS Divergence for a Multivariate Gaussian

By using the result above, we can formulate the JS divergence and thus distance in terms of the mean and covariance of a pair of multivariate Gaussians ($P$ and $Q$).

Recall that
\begin{align*}
    D_{JS}(P~||~Q) ={}& \frac{1}{2} D_{KL}(P~||~M) + \frac{1}{2} D_{KL}(Q~||~M)\\[0.6cm]
\end{align*}

Calculating $\mu_{M}$ and $\Sigma_{M}$ for the midpoint distribution, and substituting the above result into our previous definition for the JS divergence gives us

\begin{align*}
	={}& \frac{1}{2} \Bigg( \frac{1}{2} \left ( \log \frac{| \Sigma_M |}{| \Sigma_P |} - d + (\mu_M - \mu_P)^{T} \Sigma^{-1}_M (\mu_M - \mu_P) + \mathrm{Tr}(\Sigma^{-1}_M \Sigma_P) \right ) \Bigg) + \\ {}& \frac{1}{2} \Bigg( \frac{1}{2} \left ( \log \frac{| \Sigma_M |}{| \Sigma_Q |} - d + (\mu_M - \mu_Q)^{T} \Sigma^{-1}_M (\mu_M - \mu_Q) + \mathrm{Tr}(\Sigma^{-1}_M \Sigma_Q) \right ) \Bigg) \\[0.6cm]
	={}& \frac{1}{4} \left ( \log \frac{| \Sigma_M |}{| \Sigma_P |} - d + (\mu_M - \mu_P)^{T} \Sigma^{-1}_M (\mu_M - \mu_P) + \mathrm{Tr}(\Sigma^{-1}_M \Sigma_P) \right ) + \\ {}& \frac{1}{4} \left ( \log \frac{| \Sigma_M |}{| \Sigma_Q |} - d + (\mu_M - \mu_Q)^{T} \Sigma^{-1}_M (\mu_M - \mu_Q) + \mathrm{Tr}(\Sigma^{-1}_M \Sigma_Q) \right )
\end{align*}

To obtain the JS distance, we simply take the square root of this result.

# Python Implementation

Now that we have the maths, we can implement this in Python. We can then compare it by computing the JS divergence using it's entropy formulation and scipy's built-in function for the Shannon entropy.

I'll be using two simple two-dimensional Gaussians for this, defined as follows for two distributions, $P$ and $Q$

\begin{align*}
\mu_{P} = [1.1,0.5],~~~
\Sigma_{P} = 
\begin{bmatrix}
2.0 & 1.5 \\ 
1.5 & 2.5
\end{bmatrix}
\qquad\qquad
\mu_{Q} = [0.0,0.0],~~~
\Sigma_{Q} = 
\begin{bmatrix}
4.0 & 1.3 \\ 
1.3 & 3.5
\end{bmatrix}
\end{align*}

### KL Validation
Let's start by validating our KL divergence calculation.

In [20]:
import numpy as np
from scipy import stats

same = False

mu_q = np.array([[0],[0]])
sigma_q = np.array([[4.0,1.3],[1.3,3.5]])

if same:
    mu_p = np.array([[0],[0]])
    sigma_p = np.array([[4.0,1.3],[1.3,3.5]])
else:
    mu_p = np.array([[1.1],[0.5]])
    sigma_p = np.array([[2.0,1.5],[1.5,2.5]])  

def calcMidpoint(mu_p, mu_q, sigma_p, sigma_q):
    mu_m = 0.5*(mu_p+mu_q)
    sigma_m = 0.5*(sigma_p+sigma_q)
    
    return mu_m, sigma_m

mu_m, sigma_m = calcMidpoint(mu_p, mu_q, sigma_p, sigma_q)

In [18]:
def calcKL(mu_1, sigma_1, mu_2, sigma_2):
    
    assert len(mu_1) == len(mu_2) == sigma_1.shape[0] == sigma_2.shape[0]
    
    d = len(mu_1)
    
    det_1 = np.linalg.det(sigma_1)
    det_2 = np.linalg.det(sigma_2)
    
    sigma_2_inv = np.linalg.inv(sigma_2)
    
    mu_diff = mu_2 - mu_1
    
    return 0.5*(np.log(det_2/det_1) - d + 
                (mu_diff.T.dot(sigma_2_inv).dot(mu_diff)) + 
                np.trace(sigma_2_inv.dot(sigma_1)))

To compare our KL function to scipy's, which uses the normal formulation of the KL $(\sum_{i} P(i) \log \frac{P(i)}{Q(i)})$, let's sample from our distributions and feed it into scipy's entropy function and compare the output.

In [21]:
from itertools import product

q_x = np.linspace(-15,15,1000)
q_y = np.linspace(-15,15,1000)
p_x = np.linspace(-15,15,1000)
p_y = np.linspace(-15,15,1000)

p_pdf = stats.multivariate_normal.pdf(list(product(p_x,p_y)),mean=mu_p.flatten(),cov=sigma_p)
q_pdf = stats.multivariate_normal.pdf(list(product(q_x,q_y)),mean=mu_q.flatten(),cov=sigma_q)

scipy_kl = stats.entropy(pk=p_pdf, qk=q_pdf)
own_kl = calcKL(mu_p, sigma_p, mu_q, sigma_q)

print(scipy_kl, own_kl)

0.436042394844 [[ 0.43604239]]


### JS Divergence Calculation

The KL part looks fine, so let's compute the JS divergence (or distance as we return the square root). To avoid calculating the inverse and determinant twice, we'll modify the functions a little bit.

In [5]:
def calcKL_PM(mu_p, mu_m, sigma_p, det_m, inv_m):
    d = len(mu_p)
    det_p = np.linalg.det(sigma_p)
    mu_diff = mu_m - mu_p
    
    return 0.5*(np.log(det_m/det_p) - d + 
                (mu_diff.T.dot(inv_m).dot(mu_diff)) + 
                np.trace(inv_m.dot(sigma_p)))  

def calcJS(mu_p, mu_q, sigma_p, sigma_q):
    mu_m, sigma_m = calcMidpoint(mu_p, mu_q, sigma_p, sigma_q)
    
    det_m = np.linalg.det(sigma_m)
    inv_m = np.linalg.inv(sigma_m)
    
    kl_pm = calcKL_PM(mu_p, mu_m, sigma_p, det_m, inv_m)
    kl_qm = calcKL_PM(mu_q, mu_m, sigma_q, det_m, inv_m)
    
    return np.sqrt((0.5*kl_pm)+(0.5*kl_qm))

### Symmetric Validation

We'll check that it is symmetric so that it satisfies that condition. The final property that $KL$ does not have that $JS$ needs is the triangle inequality, which is better shown mathematically (the proof of this can be found in [Endres & Schindelin's paper - *A New Metric for Probability Distributions* ](https://research-repository.st-andrews.ac.uk/bitstream/handle/10023/1591/Endres2003-IEEETransInfTheory49-NewMetric.pdf;jsessionid=975E5DEE2D435B1363DEE20C2A33F439?sequence=1)

In [6]:
assert calcJS(mu_p, mu_q, sigma_p, sigma_q) == calcJS(mu_q, mu_p, sigma_q, sigma_p)
print(calcJS(mu_p, mu_q, sigma_p, sigma_q), calcJS(mu_q, mu_p, sigma_q, sigma_p))

[[ 0.33262859]] [[ 0.33262859]]
